In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-09 21:31:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-08-09 21:32:00 (11.3 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://s3.us-east-2.amazonaws.com/aws.nw.bootcamp.0805/amazon_reviews_us_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col,when,count
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
# Read in the Review dataset as a DataFrame


In [6]:
# Create the vine_table. DataFrame
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame to retrieve rows where the total_votes count is equal to or greater than 20
filtered_votes_df = vine_table.filter(df["total_votes"]>=20)
filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
filtered_votes_df.createOrReplaceTempView("filtered_votes_table")
effective_votes = spark.sql("select ROUND(CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT),2) AS useful_votes from \
filtered_votes_table where CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5 " )
effective_votes.show(5)

+------------+
|useful_votes|
+------------+
|        0.62|
|         0.6|
|        0.84|
|        0.92|
|        0.78|
+------------+
only showing top 5 rows



In [9]:
import pyspark.sql.functions as f
filtered_votes_df= filtered_votes_df.withColumn("useful_votes", f.round((filtered_votes_df.helpful_votes/filtered_votes_df.total_votes),2))
filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0.62|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|        0.45|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0.92|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0.78|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0.86|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0.56|

In [10]:
# Retrieve rows where the number of helpful_votes divided by total_votes is equal to or greater than 50% (sql query)
filtered_votes_df.createOrReplaceTempView("filtered_votes_table")
helpful_votes_df = spark.sql("select * from filtered_votes_table where CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0.62|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0.92|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0.78|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0.86|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0.56|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|         0.9|

In [11]:
# Retrieve rows where the number of helpful_votes divided by total_votes is equal to or greater than 50% (filter function)
helpful_votes_df = filtered_votes_df.filter(filtered_votes_df["useful_votes"]>=0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0.62|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0.92|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0.78|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0.86|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0.56|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|         0.9|

In [12]:
# number of vine review (from the filtered dataset)
vine_reviews_count = helpful_votes_df.where(helpful_votes_df.vine == 'Y').count()
vine_reviews_count

94

In [13]:
# number of vine review (from the filtered dataset)
non_vine_reviews_count = helpful_votes_df.where(helpful_votes_df.vine == 'N').count()
non_vine_reviews_count

40473

In [14]:
# Retrieve rows where a review was written as part of the Vine program (paid), vine == 'Y'
paid_vine_df = helpful_votes_df.filter(helpful_votes_df["vine"]=='Y')
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|        0.89|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|         1.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|        0.79|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|        0.85|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|        0.77|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|         0.9|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|        0.79|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|        0.88|

In [15]:
# Retrieve rows where a review was not written as part of the Vine program (unpaid), vine == 'N'
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df["vine"]=='N')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0.62|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0.92|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0.78|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0.86|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0.56|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|         0.9|

In [16]:
# Count total reviews in the filtered dataframe
filtered_reviews_count = helpful_votes_df.count()
filtered_reviews_count

40567

In [17]:
# count 5-star reviews from the filtered dataframe
five_star_count = helpful_votes_df.where(helpful_votes_df.star_rating == 5).count()
five_star_count

15712

In [18]:
# Count unpaid 5-star reviews in the filtered dataframe
unpaid_reiews_count = helpful_votes_df.select("*").where((helpful_votes_df.star_rating==5) & (helpful_votes_df.vine=='N')).count()
unpaid_reiews_count

15664

In [19]:
# Count paid 5-star reviews in the filtered dataframe
paid_reiews_count = helpful_votes_df.select("*").where((helpful_votes_df.star_rating==5) & (helpful_votes_df.vine=='Y')).count()
paid_reiews_count

48

In [20]:
# Percentage of 5-star reviews that is unpaid
unpaid_review_percentage = round((helpful_votes_df.select("*").where((helpful_votes_df.star_rating==5) & \
                                    (helpful_votes_df.vine=='N')).count()/helpful_votes_df.where(helpful_votes_df.vine == 'N').count())*100,2)
unpaid_review_percentage

38.7

In [21]:
helpful_votes_df.createOrReplaceTempView("helpful_votes_table")
effective_votes = spark.sql("SELECT ROUND(100.0* x.pcount/y.pcount,2) AS effective_paid FROM\
(SELECT COUNT(*) AS pcount  FROM helpful_votes_table WHERE star_rating = 5 AND vine = 'Y') x JOIN\
(SELECT COUNT(*) AS pcount FROM helpful_votes_table WHERE vine = 'Y') y on 1=1" )
effective_votes.show()

+--------------+
|effective_paid|
+--------------+
|         51.06|
+--------------+



In [22]:
# Percentage of 5-star reviews that is paid
paid_review_percentage = round((helpful_votes_df.select("*").where((helpful_votes_df.star_rating==5) & \
                                    (helpful_votes_df.vine=='Y')).count()/helpful_votes_df.where(helpful_votes_df.vine == 'Y').count())*100,2)
paid_review_percentage

51.06

In [23]:
# table to hold the review analysis
review_analysis_df = spark.createDataFrame([
                                   (filtered_reviews_count, five_star_count, unpaid_review_percentage, paid_review_percentage)
                                   
], ["Total Reviews", "Number of 5-star Reviews", "Percentage of Paid", "Percentage of Unpaid"])
review_analysis_df.show()

+-------------+------------------------+------------------+--------------------+
|Total Reviews|Number of 5-star Reviews|Percentage of Paid|Percentage of Unpaid|
+-------------+------------------------+------------------+--------------------+
|        40567|                   15712|              38.7|               51.06|
+-------------+------------------------+------------------+--------------------+

